In [1]:
# following along https://www.youtube.com/watch?v=kCc8FmEb1nY&t=1516s
# this is a custom data that I created

with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [2]:
print("length of the characters: ", len(text))

length of the characters:  465151


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)

	
 ,-;=ABCDEFGHIJKLMNOPRSTUVYZ[]abcdefghijklmnoprstuvyzÂÇÖÜâçîöûüğİıŞş‘’
73


In [4]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

In [5]:
encode = lambda s: [stoi[c] for c in s] # take a string, output a list of integers
decode = lambda l: "".join([itos[i] for i in l]) # take a list of integers, output string

In [6]:
print(encode("merhaba"))
print(decode(encode("merhaba")))

[45, 37, 49, 40, 33, 34, 33]
merhaba


In [7]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([465151]) torch.int64
tensor([15, 37, 49,  3, 70, 37, 54, 36, 37, 46,  3, 63, 46, 35, 37,  3, 39, 37,
        44, 37, 46,  3, 34, 41, 49,  3, 47, 44, 33, 46,  3,  8, 44, 44, 33, 40,
        72, 51, 68, 49,  1, 25, 47, 46, 50, 52, 55,  3, 40, 33, 45, 36,  3, 53,
        37,  3, 63, 53, 39, 65,  3, 33, 46, 35, 33, 43,  3, 47, 46, 33,  3, 54,
        33, 49, 33, 70, 68, 49,  1, 27, 44, 52, 44, 52, 43,  3, 50, 33, 40, 41,
        34, 41,  4,  3, 39, 65, 61, 44, 65,  4,  3, 55, 65,  5,  5, 35, 37, 44,
        33, 44,  1, 29, 33, 49, 33, 51, 33, 46,  4,  3, 51, 65, 49, 37, 51, 37,
        46,  3, 53, 37,  3, 43, 33, 36, 41, 49,  3, 43, 37, 45, 33, 44,  1, 29,
        37, 49, 41, 46,  4,  3, 39, 63, 66, 65, 46,  3, 53, 37,  3, 54, 33, 49,
        33, 51, 68, 44, 45, 68, 70,  3, 47, 44, 33, 46, 44, 33, 49, 68, 46,  1,
        24, 68, 55, 43, 68, 46, 68,  3, 40, 33, 55, 68, 49, 44, 33, 45, 68, 70,
        51, 68, 49,  4,  3, 50, 37, 46,  3, 39, 65, 44, 37,  3, 39, 65, 44, 37,
       

In [8]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([15, 37, 49,  3, 70, 37, 54, 36, 37])

In [10]:
x = train_data[: block_size]
y = train_data[1: block_size+1]

for t in range(block_size):
    context = x[: t+1]
    target = y[t]

    print(f"when input is {context}, the target is: {target}")

when input is tensor([15]), the target is: 37
when input is tensor([15, 37]), the target is: 49
when input is tensor([15, 37, 49]), the target is: 3
when input is tensor([15, 37, 49,  3]), the target is: 70
when input is tensor([15, 37, 49,  3, 70]), the target is: 37
when input is tensor([15, 37, 49,  3, 70, 37]), the target is: 54
when input is tensor([15, 37, 49,  3, 70, 37, 54]), the target is: 36
when input is tensor([15, 37, 49,  3, 70, 37, 54, 36]), the target is: 37


In [15]:
torch.manual_seed(1337)

batch_size = 4 # the number of the independent sequences that will be processed in parallel
block_size = 8 # the number of he maximum context length

def get_batch(split):
    # generate a small batch of data of inputs x and target y
    data = train_data if split == "train" else val_data

    ix = torch.randint(
        len(data) - block_size,
        (batch_size,)
    )

    x = torch.stack(
        [data[i: i+block_size] for i in ix]
    )
    y = torch.stack(
        [data[i+1: i+block_size+1] for i in ix]
    )

    return x, y

xb, yb = get_batch("train")
print("inputs: ")
print(xb.shape)
print(xb)
print("targets: ")
print(yb.shape)
print(yb)

inputs: 
torch.Size([4, 8])
tensor([[37,  3, 36, 68, 70, 51, 33, 46],
        [43, 68, 48,  1, 30, 41, 54, 33],
        [33,  4,  3, 46, 33, 50, 68, 44],
        [46, 41,  1, 57, 68, 43, 33, 49]])
targets: 
torch.Size([4, 8])
tensor([[ 3, 36, 68, 70, 51, 33, 46,  3],
        [68, 48,  1, 30, 41, 54, 33, 49],
        [ 4,  3, 46, 33, 50, 68, 44,  3],
        [41,  1, 57, 68, 43, 33, 49, 45]])


In [16]:
print(xb) # our input to the transformer

tensor([[37,  3, 36, 68, 70, 51, 33, 46],
        [43, 68, 48,  1, 30, 41, 54, 33],
        [33,  4,  3, 46, 33, 50, 68, 44],
        [46, 41,  1, 57, 68, 43, 33, 49]])


In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BiagramLanguageModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets:
            B, T, C = logits.shape
            # cross_entropy işlevinin girdiyi kabul etmesi için tensorları birleştiriyoruz.
            # bu aynı zamanda logits.view(-1) olarak da yapılabilir.
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits, targets)
        else:
            loss = None

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self.forward(idx)
            # focus only the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim = 1) # (B, C)
            # sample from the distrubition
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BiagramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)

logits shape: torch.Size([4, 8, 73])
targets shape: torch.Size([4, 8])
torch.Size([32, 73])


In [20]:
embedding = nn.Embedding(vocab_size, vocab_size)
logits = embedding(xb)

In [23]:
logits.shape

torch.Size([4, 8, 73])

In [24]:
logits[:, -1, :].shape

torch.Size([4, 73])